<a href="https://colab.research.google.com/github/HuyHCMUS/Digit-Regconizer/blob/main/Digit_Regconize_NN_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
import pandas as pd
import numpy as np

In [3]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score


In [4]:
df = pd.read_csv('train.csv')
df

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data = df.values
y = data[:,0]
X = data[:,1:]

y_val = torch.from_numpy(y[:1000])
y_train = torch.from_numpy(y[1000:])

X_val = torch.from_numpy(X[:1000,:]).type(torch.float)
X_train = torch.from_numpy(X[1000:,:]).type(torch.float)

X_train /=255.0
X_val /= 255.0

In [6]:
batch_size = 64

In [7]:
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [8]:
class Simple_NN(nn.Module):
    def __init__(self,input_features,output_features, hidden_units=16):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=input_features, out_features=hidden_units)
        self.layer_2 = nn.Linear(in_features=hidden_units, out_features=hidden_units)
        self.layer_3 = nn.Linear(in_features=hidden_units, out_features=output_features)
        self.relu = nn.ReLU()

    def forward(self, x):
      x = self.relu(self.layer_1(x))
      x = self.relu(self.layer_2(x))
      x = self.layer_3(x)
      return x


In [9]:
model = Simple_NN(X_train.shape[1],10,64).to(device)
model

Simple_NN(
  (layer_1): Linear(in_features=784, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_3): Linear(in_features=64, out_features=10, bias=True)
  (relu): ReLU()
)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)

In [11]:
num_epochs = 10
for epoch in range(num_epochs):
  model.train()
  for x, y in train_loader:
    x = x.to(device)
    y = y.to(device)

    output = model(x)
    loss = criterion(output,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  model.eval()
  with torch.inference_mode():
    y_preds = []
    for x, y in val_loader:
      x = x.to(device)
      y = y.to(device)

      output = model(x).cpu().numpy()
      y_preds.extend(np.argmax(output, axis = 1))
    accuracy = accuracy_score(y_val, y_preds)
    #print(y_val,y_preds)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {accuracy * 100:.2f}%')


Epoch [1/10], Validation Accuracy: 93.10%
Epoch [2/10], Validation Accuracy: 93.20%
Epoch [3/10], Validation Accuracy: 94.00%
Epoch [4/10], Validation Accuracy: 96.90%
Epoch [5/10], Validation Accuracy: 96.80%
Epoch [6/10], Validation Accuracy: 97.60%
Epoch [7/10], Validation Accuracy: 97.40%
Epoch [8/10], Validation Accuracy: 97.70%
Epoch [9/10], Validation Accuracy: 97.50%
Epoch [10/10], Validation Accuracy: 98.10%
